In [46]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn import over_sampling

In [17]:
ca_data = pd.read_csv('data/CAvideos.csv', sep=',')
gb_data = pd.read_csv('data/GBvideos.csv', sep=',')
us_data = pd.read_csv('data/USvideos.csv', sep=',')

In [18]:
ca_data['country']='Canada'
gb_data['country']='UK'
us_data['country']='US'

In [25]:
frames = [ca_data, gb_data, us_data]
youtube_data = pd.concat(frames)
youtube_data

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,Canada
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,Canada
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Canada
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Canada
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,The Cat Who Caught the Laser,AaronsAnimals,15,2018-05-18T13:00:04.000Z,"aarons animals|""aarons""|""animals""|""cat""|""cats""...",1685609,38160,1385,2657,https://i.ytimg.com/vi/BZt0qjTWNhw/default.jpg,False,False,False,The Cat Who Caught the Laser - Aaron's Animals,US
40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,22,2018-05-18T01:00:06.000Z,[none],1064798,60008,382,3936,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,NaN,US
40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,24,2018-05-18T17:34:22.000Z,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...,US
40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,1,2018-05-17T17:00:04.000Z,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...,US


In [26]:
youtube_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120746 entries, 0 to 40948
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                120746 non-null  object
 1   trending_date           120746 non-null  object
 2   title                   120746 non-null  object
 3   channel_title           120746 non-null  object
 4   category_id             120746 non-null  int64 
 5   publish_time            120746 non-null  object
 6   tags                    120746 non-null  object
 7   views                   120746 non-null  int64 
 8   likes                   120746 non-null  int64 
 9   dislikes                120746 non-null  int64 
 10  comment_count           120746 non-null  int64 
 11  thumbnail_link          120746 non-null  object
 12  comments_disabled       120746 non-null  bool  
 13  ratings_disabled        120746 non-null  bool  
 14  video_error_or_removed  120746 non-nu

In [27]:
numerical = ['likes', 'dislikes', 'views', 'comment_count']
youtube_data[numerical].describe()

,likes,dislikes,views,comment_count
count,1.207460e+05,1.207460e+05,1.207460e+05,1.207460e+05
mean,8.194300e+04,4.392414e+03,3.094371e+06,8.790323e+03
std,2.544091e+05,3.536104e+04,1.195077e+07,3.835251e+04
min,0.000000e+00,0.000000e+00,5.490000e+02,0.000000e+00
25%,3.996000e+03,1.540000e+02,1.957358e+05,5.510000e+02
50%,1.545050e+04,5.230000e+02,5.915810e+05,1.750000e+03
75%,5.657850e+04,1.879000e+03,1.842280e+06,5.707000e+03
max,5.613827e+06,1.944971e+06,4.245389e+08,1.626501e+06


In [44]:
unique_tags = [val.strip() for sublist in youtube_data.tags.dropna().str.split("|").tolist() for val in sublist]

In [48]:
unique_tags

['Eminem',
 '"Walk"',
 '"On"',
 '"Water"',
 '"Aftermath/Shady/Interscope"',
 '"Rap"',
 'plush',
 '"bad unboxing"',
 '"unboxing"',
 '"fan mail"',
 '"idubbbztv"',
 '"idubbbztv2"',
 '"things"',
 '"best"',
 '"packages"',
 '"plushies"',
 '"chontent chop"',
 'racist superman',
 '"rudy"',
 '"mancuso"',
 '"king"',
 '"bach"',
 '"racist"',
 '"superman"',
 '"love"',
 '"rudy mancuso poo bear black white official music video"',
 '"iphone x by pineapple"',
 '"lelepons"',
 '"hannahstocking"',
 '"rudymancuso"',
 '"inanna"',
 '"anwar"',
 '"sarkis"',
 '"shots"',
 '"shotsstudios"',
 '"alesso"',
 '"anitta"',
 '"brazil"',
 '"Getting My Driver\'s License',
 'Lele Pons"',
 'ryan',
 '"higa"',
 '"higatv"',
 '"nigahiga"',
 '"i dare you"',
 '"idy"',
 '"rhpc"',
 '"dares"',
 '"no truth"',
 '"comments"',
 '"comedy"',
 '"funny"',
 '"stupid"',
 '"fail"',
 'edsheeran',
 '"ed sheeran"',
 '"acoustic"',
 '"live"',
 '"cover"',
 '"official"',
 '"remix"',
 '"official video"',
 '"lyrics"',
 '"session"',
 '#DramaAlert',
 '"Dr

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

